In [1]:
!python -V

Python 3.9.21


In [2]:
import pandas as pd
import pickle
import seaborn as sns
import matplotlib.pyplot as plt

In [3]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge

from sklearn.metrics import root_mean_squared_error

In [ ]:
import mlflow
mlflow.set_tracking_uri("sqlite:///mlflow.db")
mlflow.set_experiment("nyc-taxi-experiment") 

<Experiment: artifact_location='/Users/songlisakim/workspace/mlops-learning/mlops-zoomcamp/02-experiment-tracking/mlruns/1', creation_time=1748522465340, experiment_id='1', last_update_time=1748522465340, lifecycle_stage='active', name='nyc-taxi-experiment', tags={}>

In [ ]:
def read_dataframe(filename):
    if filename.endswith('.csv'):
        df = pd.read_csv(filename)

        df.lpep_dropoff_datetime = pd.to_datetime(df.lpep_dropoff_datetime)
        df.lpep_pickup_datetime = pd.to_datetime(df.lpep_pickup_datetime)
    elif filename.endswith('.parquet'):
        df = pd.read_parquet(filename)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df



In [6]:
train_data_path = './data/green_tripdata_2021-01.parquet'
val_data_path = './data/green_tripdata_2021-02.parquet'
df_train = read_dataframe(train_data_path)
df_val = read_dataframe(val_data_path)

In [7]:
len(df_train), len(df_val)

(73908, 61921)

In [8]:
df_train['PU_DO'] = df_train['PULocationID'] + '_' + df_train['DOLocationID']
df_val['PU_DO'] = df_val['PULocationID'] + '_' + df_val['DOLocationID']

In [9]:
categorical = ['PU_DO'] #'PULocationID', 'DOLocationID']
numerical = ['trip_distance']

dv = DictVectorizer()

train_dicts = df_train[categorical + numerical].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

val_dicts = df_val[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(val_dicts)

In [10]:
target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

In [11]:
lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_val)

root_mean_squared_error(y_val, y_pred)

7.758715209766079

In [12]:
import os

def ensure_directory_exists(directory_path):
    if not os.path.exists(directory_path):
        os.makedirs(directory_path)
        print(f"Directory created: {directory_path}")
    else:
        print(f"Directory already exists: {directory_path}")


In [13]:
model_dir='models'
ensure_directory_exists(model_dir)

model_path = f'{model_dir}/lin_reg.bin'

with open(model_path, 'wb') as f_out:
    pickle.dump((dv, lr), f_out)

Directory already exists: models


In [14]:
with mlflow.start_run():
    # Parameters
    alpha = 0.01
    
    # Training
    lr = Lasso(alpha)
    lr.fit(X_train, y_train)

    # Inference
    y_pred = lr.predict(X_val)
    
    # Evaluation
    rmse = root_mean_squared_error(y_val, y_pred)
    
    mlflow.set_tag("developer", "songeun")
    mlflow.log_param("train-data-path", train_data_path)
    mlflow.log_param("valid-data-path", val_data_path)
    mlflow.log_param("alpha", alpha)
    mlflow.log_metric("rmse", rmse)
    mlflow.log_artifact(local_path=model_path, artifact_path="models_pickle")



In [17]:
import xgboost as xgb

# hyperopt is a library which uses bbayesian methods to find best hyperparam.
# fmin: to minimize output cost
# tpe: algorithm
# hp: contains different methods to define search space
# STATUS_ok: signal to send at the end of each run
# Trials: keep track of info from each run
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.pyll import scope


In [20]:
train = xgb.DMatrix(X_train, label=y_train)
valid = xgb.DMatrix(X_val, label=y_val)


In [21]:
def objective(params):
    with mlflow.start_run():
        mlflow.set_tag("model", "xgboost")
        mlflow.log_params(params)
        booster = xgb.train(
            params=params,
            dtrain=train,
            num_boost_round=1000,
            evals=[(valid, "validation")],
            early_stopping_rounds=50
        )
        y_pred = booster.predict(valid)
        rmse = root_mean_squared_error(y_val, y_pred)
        mlflow.log_metric("rmse", rmse)
    return {'loss': rmse, 'status': STATUS_OK}

In [61]:
from math import exp
exp(-3)

0.049787068367863944

In [ ]:
params = {
    'learning_rate': 0.08689277546104228,
    'max_depth': 41,
    'min_child_weight': 1.0573384280854001,
    'objective': 'reg:linear',
    'reg_alpha': 0.019777208733018477,
    'reg_lambda': 0.03147085383119916,
    'seed': 42
}

mlflow.xgboost.autolog()
booster = xgb.train(
    params=params,
    dtrain=train,
    num_boost_round=1000,
    evals=[(valid, "validation")],
    early_stopping_rounds=50
)


In [33]:
mlflow.xgboost.autolog(disable=False)

In [27]:
logged_model = 'runs:/e7dc46a4a7d24cecab0ea999910280f5/models_mlflow'

# Load model as a PyFuncModel
loaded_model = mlflow.pyfunc.load_model(logged_model)


In [28]:
loaded_model

mlflow.pyfunc.loaded_model:
  artifact_path: models_mlflow
  flavor: mlflow.xgboost
  run_id: e7dc46a4a7d24cecab0ea999910280f5

In [29]:
import mlflow.xgboost


xgboost_model = mlflow.xgboost.load_model(logged_model)

In [30]:
xgboost_model

In [31]:
y_pred = xgboost_model.predict(valid)

In [32]:
y_pred[:10]

array([14.425333 ,  7.0882463, 15.602779 , 24.381384 ,  9.47145  ,
       17.147675 , 11.27457  ,  8.200635 ,  9.010728 , 19.639137 ],
      dtype=float32)